In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# generate a sequence of numbers
def generate_sequence():
    return [i for i in range(1, 450, 7)]

In [3]:
# one hot encode sequence
def one_hot_encode(sequence, n_unique = 450):
    encoding = list()
    for value in sequence:
        vector = [0 for i in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return np.array(encoding)

In [4]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [5]:
# generate data for the lstm
def generate_data():
    # generate sequence
    sequence = generate_sequence()
    
    # one hot encode
    encoded = one_hot_encode(sequence)
    
    # create lag inputs
    df = pd.DataFrame(encoded)
    df = pd.concat([df.shift(4), df.shift(3), df.shift(2), df.shift(1), df], axis = 1)
    
    # remove non-viable rows
    values = df.values
    values = values[5:,:]
    
    # convert to 3d for input
    X = values.reshape(len(values), 5, 450)
    
    # drop last value from y
    y = encoded[4:-1,:]
    return X, y

In [6]:
# define model
model = Sequential()
model.add(LSTM(50, batch_input_shape = (5, 5, 450), stateful = True))
model.add(Dense(450, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [7]:
# fit model
for i in range(150):
    X, y = generate_data()
    model.fit(X, y, epochs = 1, batch_size = 5, verbose = 1, shuffle = False)
    model.reset_states()

Epoch 1/1
60/60 [==============================] - 1s 10ms/step - loss: 6.1113 - acc: 0.0000e+00
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 6.0851 - acc: 0.0167
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 6.0567 - acc: 0.0500
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 6.0039 - acc: 0.1000
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 5.6559 - acc: 0.0500
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 4.9549 - acc: 0.0333
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 4.4870 - acc: 0.0167
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 4.2998 - acc: 0.0167
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 4.2266 - acc: 0.0167
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 4.1708 - acc: 0.0167
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 4.12

60/60 [==============================] - 0s 2ms/step - loss: 2.3566 - acc: 0.8500
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.3483 - acc: 0.7500
Epoch 1/1
60/60 [==============================] - 0s 3ms/step - loss: 2.3387 - acc: 0.7167
Epoch 1/1
60/60 [==============================] - 0s 3ms/step - loss: 2.3528 - acc: 0.8000
Epoch 1/1
60/60 [==============================] - 0s 3ms/step - loss: 2.2863 - acc: 0.8000
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.2696 - acc: 0.8333
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.2526 - acc: 0.8667
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.2342 - acc: 0.8667
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.2258 - acc: 0.8667
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.2146 - acc: 0.8500
Epoch 1/1
60/60 [==============================] - 0s 2ms/step - loss: 2.2207 - acc: 0.800

In [8]:
# evaluate model on new data
X, y = generate_data()
yhat = model.predict(X, batch_size = 5)

print(f'Expected: {one_hot_decode(y)}')
print("")
print(f'Predicted: {one_hot_decode(yhat)}')

Expected: [29, 36, 43, 50, 57, 64, 71, 78, 85, 92, 99, 106, 113, 120, 127, 134, 141, 148, 155, 162, 169, 176, 183, 190, 197, 204, 211, 218, 225, 232, 239, 246, 253, 260, 267, 274, 281, 288, 295, 302, 309, 316, 323, 330, 337, 344, 351, 358, 365, 372, 379, 386, 393, 400, 407, 414, 421, 428, 435, 442]

Predicted: [29, 36, 43, 50, 57, 64, 71, 78, 85, 92, 99, 106, 113, 120, 127, 134, 141, 148, 155, 162, 169, 176, 183, 190, 197, 204, 211, 218, 225, 232, 239, 246, 253, 260, 267, 274, 281, 288, 295, 302, 309, 316, 323, 330, 337, 344, 351, 358, 365, 372, 379, 386, 393, 400, 407, 414, 421, 428, 435, 442]


In [9]:
y == yhat

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])